In [ ]:
#API
import requests

#ETL
import pandas as pd
from pandas import json_normalize
import time
import json
import numpy as np
from datetime import datetime, timezone
from pathlib import Path

#DB
import psycopg2
from psycopg2.extras import Json
from sqlalchemy import create_engine, text
from supabase import create_client, Client
from sqlalchemy.exc import SQLAlchemyError

#Other
import os
from dotenv import load_dotenv

In [22]:
load_dotenv()
api_key = os.getenv('api_key')
db_password = os.getenv('db_password')
db_user = os.getenv('db_user')
db_host = os.getenv('db_host')
db_port = os.getenv('db_port')
db_name = os.getenv('db_name')

In [ ]:
headers = {"Authorization": f"Bearer {api_key}"}

In [3]:
# All league identifiers

#S54
advanced_id = '786da1fa-a9b5-4d17-b40e-416588272db4'
main_id = 'bd7957b3-1114-435f-af43-22645a508cc0'
intermediate_id ='4410aa73-9584-4500-a7ca-1504ed6da707'
entry_id ='e61be909-1727-4de3-8201-419d89893401'
open10_id ='f62c4c75-f620-40a2-bef8-0be2cb85d2d3'
open9_id = '0ce757ef-9bad-4afa-bbce-58d647824222'
open1_8_id ='3cd81a7e-ed93-43ba-be08-2a1ee7ad7237'

region= '55a91ca4-fa67-48b1-aed7-ff635fecc3ed'

championship_ids = ['786da1fa-a9b5-4d17-b40e-416588272db4',
                    'bd7957b3-1114-435f-af43-22645a508cc0',
                    '4410aa73-9584-4500-a7ca-1504ed6da707',
                    'e61be909-1727-4de3-8201-419d89893401',
                    'f62c4c75-f620-40a2-bef8-0be2cb85d2d3',
                    '0ce757ef-9bad-4afa-bbce-58d647824222',
                    '3cd81a7e-ed93-43ba-be08-2a1ee7ad7237']

## Call API pour récupérer les données

### Détail des leagues

In [4]:
## api call not working for some reason

### List of match played per championships

In [4]:

def fetch_and_save_all_matches(championship_ids, headers, filename="all_matches.json", limit=100, delay=0.3):

    def get_all_matches_for_champ(championship_id):
        offset = 0
        champ_matches = []
        while True:
            url = f"https://open.faceit.com/data/v4/championships/{championship_id}/matches?region={region}&type=past&limit={limit}&offset={offset}"
            response = requests.get(url, headers=headers)

            if response.status_code != 200:
                print(f"Error for {championship_id} at offset {offset}: Status Code {response.status_code} - {response.text}")
                break

            data = response.json()
            items = data.get("items", [])

            if not items:
                print(f"No more matches found for championship {championship_id} at offset {offset}")
                break

            champ_matches.extend(items)
            print(f"Fetched {len(items)} matches for championship {championship_id} at offset {offset}")

            offset += limit
            time.sleep(delay)

        return champ_matches
    
    all_matches= []

    for champ_id in championship_ids:
        print(f"Fetching matches for championship: {champ_id}")
        matches = get_all_matches_for_champ(champ_id)
        print(f"Total matches found for championship {champ_id}: {len(matches)}")
        all_matches.extend(matches)

    with open(filename, "w") as f:
        json.dump(all_matches, f, indent=2)

    print(f"Total matches collected across all championships: {len(all_matches)}")

    return all_matches

In [8]:
#Using API or local  cached file

#all_matches = fetch_and_save_all_matches(championship_ids, headers,filename="all_matches.json", limit=100, delay=0.3)

with open("all_matches.json", "r", encoding="utf-8") as f:
    all_matches = json.load(f)


In [9]:
match_list = [match['match_id'] for match in all_matches]

### Details of each match listed

In [7]:
def fetch_match_stats(match_list, headers, stats_file="match_stats.json", no_stats_file="no_stats.json", delay=0.2):
    stats_path = Path(stats_file)
    no_stats_path = Path(no_stats_file)

    # 1. Charger les matchs avec stats existants
    if stats_path.exists():
        with open(stats_path, "r") as f:
            all_stats = json.load(f)
    else:
        all_stats = {}

    # 2. Charger matchs sans stats
    if no_stats_path.exists():
        with open(no_stats_path, "r") as f:
            no_stats = set(json.load(f))
    else:
        no_stats = set()

    # 3. Calculer les matchs à récupérer
    match_set = set(match_list)
    already_fetched = set(all_stats.keys())
    to_skip = already_fetched.union(no_stats)
    to_fetch = list(match_set - to_skip)

    print(f"🔍 {len(to_fetch)} nouveaux matchs à récupérer parmi {len(match_list)} total.\n")

    # 4. Boucle sur matchs à récupérer
    for i, match_id in enumerate(to_fetch, start=1):
        url = f"https://open.faceit.com/data/v4/matches/{match_id}/stats"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print(f"[{i}/{len(to_fetch)}] {match_id} → Pas de stats (404) ❌")
                no_stats.add(match_id)
                continue

            response.raise_for_status()
            match_data = response.json()

            all_stats[match_id] = match_data
            print(f"[{i}/{len(to_fetch)}] {match_id} récupéré ✅")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"[{i}/{len(to_fetch)}] Erreur pour {match_id} ❌ : {e}")
            continue

    # 5. Sauvegarde finale
    with open(stats_path, "w") as f:
        json.dump(all_stats, f, indent=2)

    with open(no_stats_path, "w") as f:
        json.dump(list(no_stats), f, indent=2)

    print(f"\n✅ Terminé ! Total stockés : {len(all_stats)} ✅ | Sans stats : {len(no_stats)} ❌")


    return all_stats, no_stats


In [29]:
#Using API or local  cached file

# all_stats, no_stats = fetch_match_stats(match_list, headers, stats_file="match_stats.json", no_stats_file="no_stats.json", delay=0.2)

with open("match_stats.json", "r", encoding="utf-8") as f:
    all_stats = json.load(f)

In [31]:
team_ids = set()  # set pour éviter les doublons

for match_id, match_data in all_stats.items():
    rounds = match_data.get('rounds', [])
    for round_data in rounds:
        teams = round_data.get('teams', [])
        for team in teams:
            team_id = team.get('team_id')
            if team_id:
                team_ids.add(team_id)

team_ids = list(team_ids)

### Details of each team involved in matches related to championships

In [9]:
def fetch_team_details(team_ids, headers, team_file="team_details.json", delay=0.3):
    team_path = Path(team_file)

    # Charger les données existantes si présentes
    if team_path.exists():
        with open(team_path, "r") as f:
            all_teams = json.load(f)
    else:
        all_teams = {}

    # Liste des IDs déjà récupérés
    already_fetched = set(all_teams.keys())

    # Filtrer les team_ids à récupérer
    to_fetch = [tid for tid in team_ids if tid not in already_fetched]

    print(f"🔍 {len(to_fetch)} nouvelles équipes à récupérer parmi {len(team_ids)} au total.\n")

    for i, team_id in enumerate(to_fetch, start=1):
        url = f"https://open.faceit.com/data/v4/teams/{team_id}"

        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 404:
                print(f"[{i}/{len(to_fetch)}] {team_id} → Équipe introuvable (404) ❌")
                all_teams[team_id] = None  # Ou gérer comme tu veux
                continue

            response.raise_for_status()
            team_data = response.json()

            all_teams[team_id] = team_data
            print(f"[{i}/{len(to_fetch)}] {team_id} récupérée ✅")

            time.sleep(delay)

        except requests.exceptions.RequestException as e:
            print(f"[{i}/{len(to_fetch)}] Erreur pour {team_id} ❌ : {e}")
            continue

    # Sauvegarde finale
    with open(team_path, "w") as f:
        json.dump(all_teams, f, indent=2)

    print(f"\n✅ Terminé ! Total équipes stockées : {len(all_teams)}")

    return all_teams


In [28]:
#Using API or local  cached file

#all_teams = fetch_team_details(team_ids, headers)

with open("team_details.json", "r", encoding="utf-8") as f:
    all_teams = json.load(f)

## Tables building

### dim_championship table

In [12]:
championship_details_list = []

for data in all_matches:  # juste itérer sur la liste
    if data is None:
        continue  # skip entries with no data

    row = {
        'competition_id': data.get('competition_id'),
        'competition_name': data.get('competition_name'),
        'competition_type': data.get('competition_type'),
        'region': data.get('region'),
        'started_at': data.get('finished_at'), #on utilise la valeur de finished_at comme jour de match car si le match a été dodge il n'y a pas de start_date
    }
    championship_details_list.append(row)

dim_championships = pd.DataFrame(championship_details_list)

In [13]:
dim_championships['started_at'] = pd.to_datetime(dim_championships['started_at'], unit='s', utc=True).dt.date


In [14]:
dim_championships[['organizer','season','region','division','sub_region','state','state2','state3']] = dim_championships['competition_name'].str.split(' ',expand=True)
dim_championships['state'] = dim_championships['state2'] + ' ' + dim_championships['state3']
dim_championships.drop(columns=['state2', 'state3'], inplace=True)

In [15]:
dim_championships = dim_championships.groupby(["competition_id"]).agg({
    "organizer": "first", 
    "competition_type": "first", 
    'region':'first',
    'sub_region':'first',
    "season": "first",       
    "division": "first",               
    "state": "first",
    "started_at" :"first",
}).reset_index()

In [16]:
dim_championships

,competition_id,organizer,competition_type,region,sub_region,season,division,state,started_at
0,0ce757ef-9bad-4afa-bbce-58d647824222,ESEA,championship,EU,Central,S54,Open9,Regular Season,2025-08-08
1,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,ESEA,championship,EU,Central,S54,Open1-8,Regular Season,2025-08-10
2,4410aa73-9584-4500-a7ca-1504ed6da707,ESEA,championship,EU,Central,S54,Intermediate,Regular Season,2025-08-11
3,786da1fa-a9b5-4d17-b40e-416588272db4,ESEA,championship,EU,Central,S54,Advanced,Regular Season,2025-08-11
4,bd7957b3-1114-435f-af43-22645a508cc0,ESEA,championship,EU,Central,S54,Main,Regular Season,2025-08-08
5,e61be909-1727-4de3-8201-419d89893401,ESEA,championship,EU,Central,S54,Entry,Regular Season,2025-08-08
6,f62c4c75-f620-40a2-bef8-0be2cb85d2d3,ESEA,championship,EU,Central,S54,Open10,Regular Season,2025-08-08


### dim_teams and dim_players tables

In [16]:
team_details_list = []

for team_ids, team_data in all_teams.items():
    if team_data is None:
        continue  # 👈 Skip entries with no data

    row = {
        'team_id': team_data.get('team_id'),
        'team_nickname': team_data.get('nickname'),
        'team_name': team_data.get('name'),
        'team_avatar': team_data.get('avatar'),
        'team_faceit_url': team_data.get('faceit_url'),
    }
    team_details_list.append(row)

dim_teams = pd.DataFrame(team_details_list)
dim_teams = dim_teams.drop_duplicates(subset=['team_id'], keep='first')

In [17]:
dim_teams

,team_id,team_nickname,team_name,team_avatar,team_faceit_url
0,cfdfa687-0075-4f23-8129-28eb01b37a2a,meoWW,meow,https://distribution.faceit-cdn.net/images/e06...,https://www.faceit.com/{lang}/teams/cfdfa687-0...
1,178c3540-6145-48f4-b0c1-91454f727d17,E-Town,E-Town Gaming,https://distribution.faceit-cdn.net/images/82b...,https://www.faceit.com/{lang}/teams/178c3540-6...
2,ee7070bb-bc2e-474c-855b-b38c09b4245d,ECOLE,Ecole,None,https://www.faceit.com/{lang}/teams/ee7070bb-b...
3,c6c8e646-7aa3-4df6-89dd-e8ced6760fce,LoPu,Lost Puppies,https://distribution.faceit-cdn.net/images/6f1...,https://www.faceit.com/{lang}/teams/c6c8e646-7...
4,3b06c47e-2db4-46c7-a518-16db37d28eec,Cuc9,CuCbKa_nuBa,https://distribution.faceit-cdn.net/images/424...,https://www.faceit.com/{lang}/teams/3b06c47e-2...
...,...,...,...,...,...
1220,82bc825b-7a20-4a2f-b588-ee9026342870,Split-,Split ESC,https://distribution.faceit-cdn.net/images/975...,https://www.faceit.com/{lang}/teams/82bc825b-7...
1221,b4605ffb-d00d-48ef-827e-c316e80aa5e4,PRCTLGY,Proctology,None,https://www.faceit.com/{lang}/teams/b4605ffb-d...
1222,4b705480-dc5d-403b-a662-29f7036e1c4a,ATMO,Atmosferka,https://distribution.faceit-cdn.net/images/bbf...,https://www.faceit.com/{lang}/teams/4b705480-d...
1223,f85d66c7-395e-4e75-ac9f-36e1b9b26f5b,DsTm,Disband Team,https://distribution.faceit-cdn.net/images/a0b...,https://www.faceit.com/{lang}/teams/f85d66c7-3...


In [ ]:
players_details_list = []

for team_id, team_data in all_teams.items():
    if team_data is None or 'members' not in team_data:
        continue 

    for member in team_data['members']:
        row = {
            'member_user_id': member.get('user_id'),
            'team_id': team_data.get('team_id'),
            'member_nickname': member.get('nickname'),
            'member_country': member.get('country'),
            'member_faceit_url': member.get('faceit_url')
        }
        players_details_list.append(row)

# Create DataFrame
dim_players = pd.DataFrame(players_details_list)
dim_players = dim_players.rename(columns={'member_user_id':'player_id','member_nickname':'player_name','member_country':'player_country','member_faceit_url':'player_faceit_url'})
dim_players = dim_players.drop_duplicates(subset=['player_id'], keep='first')

In [19]:
dim_players

,player_id,team_id,player_name,player_country,player_faceit_url
0,67d3143c-013a-4dc7-9f72-5557f21c171f,cfdfa687-0075-4f23-8129-28eb01b37a2a,c0nsue1o,es,https://www.faceit.com/{lang}/players/c0nsue1o
1,57359b94-cb27-4485-bad2-e4a37dd01735,cfdfa687-0075-4f23-8129-28eb01b37a2a,kevinDburger,nl,https://www.faceit.com/{lang}/players/kevinDbu...
2,7313716a-85ff-4b80-8141-6cfd1965343d,cfdfa687-0075-4f23-8129-28eb01b37a2a,business666,ru,https://www.faceit.com/{lang}/players/business666
3,d48a9d1a-35ca-4f4f-af3a-9098769c8bb6,cfdfa687-0075-4f23-8129-28eb01b37a2a,fxckzavr,ru,https://www.faceit.com/{lang}/players/fxckzavr
4,2b949bfd-0f60-4827-9b82-f82d6b1a0e4c,cfdfa687-0075-4f23-8129-28eb01b37a2a,laperouse888,fr,https://www.faceit.com/{lang}/players/laperous...
...,...,...,...,...,...
10821,2edb3c81-58ab-4b4c-b055-ddc438681e3c,8d459341-6868-4cc8-ad31-b04aab1ae0de,NaChEWw77,bg,https://www.faceit.com/{lang}/players/NaChEWw77
10822,fd8416ea-c17d-4083-b67d-786628da55e3,8d459341-6868-4cc8-ad31-b04aab1ae0de,SME,bg,https://www.faceit.com/{lang}/players/SME
10823,60c44d2b-aee2-4b73-b075-118d401f3a71,8d459341-6868-4cc8-ad31-b04aab1ae0de,mitzera666,bg,https://www.faceit.com/{lang}/players/mitzera666
10824,e22b6591-f03a-46cf-a36a-277310493537,8d459341-6868-4cc8-ad31-b04aab1ae0de,pr0fesore,bg,https://www.faceit.com/{lang}/players/pr0fesore


### fact_matches table

The idea is to have a composite key with the combination of match_id and match_round being unique : 

- match id being the whole match (either its a Bo1, Bo3 or Bo5)
- match_round being the id of the map played (1 if a single map is played and then increase up to 5 for Bo5)

In [ ]:
fact_matches_champ_query_list = []

for data in all_matches:  
    if data is None:
        continue  

    row = {
        'match_id': data.get('match_id'),
        'competition_id': data.get('competition_id'),
        'started_at': data.get('finished_at'),
    }
    fact_matches_champ_query_list.append(row)

fact_matches_champ_query = pd.DataFrame(fact_matches_champ_query_list)
fact_matches_champ_query['started_at'] = pd.to_datetime(fact_matches_champ_query['started_at'], unit='s', utc=True).dt.date


In [35]:
fact_matches_champ_query

,match_id,competition_id,started_at
0,1-42464616-2d4c-494c-b3a5-bc60a877c9bb,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11
1,1-9ccbd554-70e7-4608-a5b7-96881f93310c,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11
2,1-9f4ee313-0819-4046-8768-e0bf09207e28,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11
3,1-d71f7556-1ff2-40c8-912e-a8c9b5517654,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11
4,1-b47a7ae8-93b6-45f9-8af2-63a10f6b056b,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11
...,...,...,...
3403,1-310f7089-629e-46bf-9290-ef3b5f977c1c,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-14
3404,1-8fe2b654-6347-4351-a1e6-805962c00a87,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-14
3405,1-fb828a5c-2379-4052-80b5-2e5809c80e27,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-14
3406,1-8cee1a24-9449-4d0f-87f8-c61b98454479,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,2025-07-14


In [36]:
fact_match_list = []

for match_id, match_data in all_stats.items():
    # Extract round data (1 row per map played)
    for round_data in match_data['rounds']:
        # Match Round Level (1 row per map)
        round_stats = round_data['round_stats']
        fact_match_list.append({
            "match_id": match_id,
            "match_round": round_data.get("match_round"),  # Directly access match_round from round_data
            "format": f"bo{round_data.get('best_of')}",  # Directly access best_of from round_data
            "map": round_stats.get("Map"),
            "rounds_played": round_stats.get("Rounds"),
            "winner_team_id": round_stats.get("Winner")
        })

fact_matches_matchquery = pd.DataFrame(fact_match_list)


In [37]:
fact_matches_matchquery

,match_id,match_round,format,map,rounds_played,winner_team_id
0,1-4a48ab31-36e8-4e24-9899-71fbcd5daa1f,1,bo1,de_anubis,22,40ee7839-d6df-4086-8c8f-ca998c9aedbf
1,1-b0b88140-048a-4824-910d-d399840e73b4,1,bo1,de_ancient,16,d73ef5c5-95da-45b2-b452-541c9540de36
2,1-b45de354-4328-4b10-9f60-84fc7379a3dd,1,bo1,de_mirage,20,fcb836d2-7311-468f-9d55-101044d526d5
3,1-3ca5ead7-d7fc-440d-82b4-5cd76b4ee4f4,1,bo1,de_nuke,19,7fe07eb1-177b-4ab4-9473-535b4469cb50
4,1-dc4669e5-4600-4250-8acc-b90a721851b8,1,bo1,de_train,23,f3b3ace1-84f1-448c-bc88-e222e3d3f085
...,...,...,...,...,...,...
2723,1-a45b6f45-5532-4775-b867-0631bcc2d2dc,1,bo1,de_overpass,23,ff02ced9-0876-468b-a7b3-6273e086cf9c
2724,1-4f2bec56-062a-41ba-9b26-81a1a8fa6991,1,bo1,de_train,23,1a4c9dfd-687f-4162-a754-3ca037ecdd95
2725,1-ed933277-da0a-4188-8177-8dcd67b7c433,1,bo1,de_inferno,23,e595923e-a6dc-4469-947c-3666a2b23901
2726,1-54804b1c-8ca9-43be-9294-3f7ffa527bb9,1,bo1,de_dust2,24,6a569bfd-d9ae-46d9-b961-1e55fb5bee48


In [38]:
fact_matches = pd.merge(fact_matches_champ_query[['competition_id','match_id','started_at']], fact_matches_matchquery, on='match_id')


In [39]:
fact_matches

,competition_id,match_id,started_at,match_round,format,map,rounds_played,winner_team_id
0,786da1fa-a9b5-4d17-b40e-416588272db4,1-9ccbd554-70e7-4608-a5b7-96881f93310c,2025-08-11,1,bo1,de_dust2,15,d96408dc-5839-48b4-aab8-82a4892c1b3e
1,786da1fa-a9b5-4d17-b40e-416588272db4,1-9f4ee313-0819-4046-8768-e0bf09207e28,2025-08-11,1,bo1,de_nuke,23,4c6e75da-fa4d-4ba5-8100-f732536e573d
2,786da1fa-a9b5-4d17-b40e-416588272db4,1-d71f7556-1ff2-40c8-912e-a8c9b5517654,2025-08-11,1,bo1,de_overpass,21,7258313d-9608-40dc-87b0-856505e811cb
3,786da1fa-a9b5-4d17-b40e-416588272db4,1-b47a7ae8-93b6-45f9-8af2-63a10f6b056b,2025-08-11,1,bo1,de_overpass,16,2cc2029e-67ad-44cc-a1eb-0d7889fd7fb8
4,786da1fa-a9b5-4d17-b40e-416588272db4,1-033accf0-3dfb-44b8-9ce3-7240bc4bc0c8,2025-08-11,1,bo1,de_nuke,24,56f41203-b58c-46f1-a0ef-60319f4965e7
...,...,...,...,...,...,...,...,...
2723,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,1-16379fdb-9ae1-41af-8f53-4d7db89ebb08,2025-07-14,1,bo1,de_ancient,24,028c8617-8f6c-4a1f-838c-751fdf9f615b
2724,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,1-310f7089-629e-46bf-9290-ef3b5f977c1c,2025-07-14,1,bo1,de_dust2,15,c5df0f2b-9180-4d31-806b-f3ac4d042601
2725,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,1-8fe2b654-6347-4351-a1e6-805962c00a87,2025-07-14,1,bo1,de_anubis,21,0b506b14-5a74-4324-88dd-4cb49798aef7
2726,3cd81a7e-ed93-43ba-be08-2a1ee7ad7237,1-fb828a5c-2379-4052-80b5-2e5809c80e27,2025-07-14,1,bo1,de_inferno,22,0ad17805-4c4c-4b27-9cfd-f9d6e74da021


In [40]:
column_order=['match_id','match_round','competition_id','started_at','format','map','rounds_played','winner_team_id']
fact_matches = fact_matches[column_order]
fact_matches = fact_matches.rename(columns={'started_at':'match_day','rounds_played':'total_rounds_played','winner_team_id':'winner_id'})
fact_matches = fact_matches.drop_duplicates(subset=['match_id','match_round'], keep='first')

In [42]:
fact_matches.head(2)

,match_id,match_round,competition_id,match_day,format,map,total_rounds_played,winner_id
0,1-9ccbd554-70e7-4608-a5b7-96881f93310c,1,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11,bo1,de_dust2,15,d96408dc-5839-48b4-aab8-82a4892c1b3e
1,1-9f4ee313-0819-4046-8768-e0bf09207e28,1,786da1fa-a9b5-4d17-b40e-416588272db4,2025-08-11,bo1,de_nuke,23,4c6e75da-fa4d-4ba5-8100-f732536e573d


### fact_players_stats table 

In [32]:
player_stats_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        map_name = round_data['round_stats'].get('Map')
        winner_id = round_data['round_stats'].get('Winner')

        for team in round_data['teams']:
            team_id = team.get("team_id")
            team_name = team["team_stats"].get("Team")
            team_win = team["team_stats"].get("Team Win")

            for player in team["players"]:
                player_id = player["player_id"]
                nickname = player["nickname"]
                stats = player["player_stats"]

                row = {
                    "match_id": match_id,
                    "match_round": round_data.get("match_round"),
                    "player_id": player_id,
                    "team_id": team_id,
                    "team_name": team_name,
                    "team_win": team_win,
                    "player_name": nickname
                }

                row.update(stats)
                player_stats_list.append(row)

fact_players_stats = pd.DataFrame(player_stats_list)
for col in fact_players_stats.columns:
    fact_players_stats[col] = pd.to_numeric(fact_players_stats[col], errors="ignore")

C:\Users\Armel\AppData\Local\Temp\ipykernel_10940\414250223.py:33: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  fact_players_stats[col] = pd.to_numeric(fact_players_stats[col], errors="ignore")


In [33]:
fact_players_stats = fact_players_stats.drop(columns={'team_name','team_id','team_win','player_name'})
#correction d'un cas ou les données d'une game sont en doublons, on garde les lignes qui ont les stats les plus importantes, exemple :https://www.faceit.com/fr/cs2/room/1-e4410b46-a6ac-4037-ad35-d7f91f8bf55f/scoreboard
players_stats_sorted = fact_players_stats.sort_values(['match_id', 'player_id', 'Kills'], ascending=[True, True, False])
fact_players_stats = players_stats_sorted.drop_duplicates(subset=['match_id', 'player_id'], keep='first')

In [33]:
fact_players_stats = fact_players_stats.drop(columns={'Result','Headshots %','K/R Ratio','K/D Ratio','Match Entry Rate','Match 1v1 Win Rate','Flash Success Rate per Match','Sniper Kill Rate per Round','Utility Usage per Round','Flashes per Round in a Match','Sniper Kill Rate per Match','Utility Damage per Round in a Match','Utility Damage Success Rate per Match','Utility Successes','Enemies Flashed per Round in a Match','Utility Success Rate per Match','Match Entry Success Rate','Match 1v2 Win Rate','Utility Enemies'})

In [34]:
fact_players_stats = fact_players_stats.rename(columns={
    'Zeus Kills':'zeus_kills',
    'Utility Count':'utility_count',
    'Double Kills':'double_kills',
    'Flash Successes':'flash_successes',
    'Quadro Kills':'quadro_kills',
    'Entry Count':'entry_count',
    'First Kills':'first_kills',
    'Flash Count':'flash_count',
    'Sniper Kills':'sniper_kills',
    'Damage':'damage',
    'Utility Damage':'utility_damage',
    'Assists':'assists',
    '1v1Count':'count_1v1',
    'Enemies Flashed':'enemies_flashed',
    'Clutch Kills':'clutch_kills',
    'Penta Kills':'ace',
    'MVPs':'mvps',
    'Deaths':'deaths',
    'Entry Wins':'entry_wins',
    'Kills':'kills',
    '1v2Wins':'wins_1v2',
    '1v1Wins':'wins_1v1',
    'Pistol Kills':'pistol_kills',
    'Knife Kills':'knife_kills',
    'ADR':'adr',
    '1v2Count':'count_1v2',
    'Triple Kills':'triple_kills',
    'Headshots':'headshots',

})

In [35]:
fact_players_stats.columns

Index(['match_id', 'match_round', 'player_id', 'sniper_kills', 'deaths',
       'count_1v1', 'pistol_kills', 'adr', 'entry_wins', 'quadro_kills', 'ace',
       'count_1v2', 'double_kills', 'enemies_flashed', 'triple_kills',
       'knife_kills', 'wins_1v2', 'zeus_kills', 'entry_count', 'mvps',
       'flash_count', 'assists', 'utility_count', 'utility_damage',
       'clutch_kills', 'wins_1v1', 'headshots', 'flash_successes', 'kills',
       'first_kills', 'damage'],
      dtype='object')

### fact_teams_stats table 

In [36]:
team_stats_list = []

for match_id, match_data in all_stats.items():
    for round_data in match_data['rounds']:
        map_name = round_data['round_stats'].get('Map')
        winner_id = round_data['round_stats'].get('Winner')
        total_round_played = round_data['round_stats'].get('Rounds')

        for team in round_data['teams']:
            team_id = team.get("team_id")
            team_name = team["team_stats"].get("Team")
            team_win = team["team_stats"].get("Team Win")

            row = {
                "match_id": match_id,
                "match_round": round_data.get("match_round"),
                "team_id": team_id,
                "winner_team_id": winner_id,
                "team_win": team_win
            }

            row.update(team["team_stats"])
            team_stats_list.append(row)

fact_teams_stats = pd.DataFrame(team_stats_list)


In [37]:
fact_teams_stats = fact_teams_stats.drop(columns={'Team','winner_team_id','Team Win','Team Headshots'})
fact_teams_stats = fact_teams_stats.rename(columns={'Final Score':'final_score','Overtime score':'overtime_score','First Half Score':'first_half_score','Second Half Score':'second_half_score'})

In [38]:
#correction d'un cas ou les données d'une game sont en doublons, on garde les lignes qui ont les stats les plus importantes, exemple :https://www.faceit.com/fr/cs2/room/1-e4410b46-a6ac-4037-ad35-d7f91f8bf55f/scoreboard
team_stats_sorted = fact_teams_stats.sort_values(['match_id', 'team_id', 'final_score'], ascending=[True, True, False])
fact_teams_stats = team_stats_sorted.drop_duplicates(subset=['match_id', 'team_id'], keep='first')

column_order=['match_id','match_round','team_id','team_win','first_half_score','second_half_score','overtime_score','final_score']
fact_teams_stats = fact_teams_stats[column_order]

In [39]:
fact_teams_stats.columns

Index(['match_id', 'match_round', 'team_id', 'team_win', 'first_half_score',
       'second_half_score', 'overtime_score', 'final_score'],
      dtype='object')

## Summary of tables & columns

In [40]:
dim_championships.columns

Index(['competition_id', 'organizer', 'competition_type', 'region',
       'sub_region', 'season', 'division', 'state', 'started_at'],
      dtype='object')

In [41]:
dim_teams.columns

Index(['team_id', 'team_nickname', 'team_name', 'team_avatar',
       'team_faceit_url'],
      dtype='object')

In [42]:
dim_players.columns

Index(['player_id', 'team_id', 'player_name', 'player_country',
       'player_faceit_url'],
      dtype='object')

In [43]:
fact_matches.columns

Index(['match_id', 'match_round', 'competition_id', 'match_day', 'format',
       'map', 'total_rounds_played', 'winner_id'],
      dtype='object')

In [44]:
fact_players_stats.columns

Index(['match_id', 'match_round', 'player_id', 'sniper_kills', 'deaths',
       'count_1v1', 'pistol_kills', 'adr', 'entry_wins', 'quadro_kills', 'ace',
       'count_1v2', 'double_kills', 'enemies_flashed', 'triple_kills',
       'knife_kills', 'wins_1v2', 'zeus_kills', 'entry_count', 'mvps',
       'flash_count', 'assists', 'utility_count', 'utility_damage',
       'clutch_kills', 'wins_1v1', 'headshots', 'flash_successes', 'kills',
       'first_kills', 'damage'],
      dtype='object')

In [45]:
fact_teams_stats.columns

Index(['match_id', 'match_round', 'team_id', 'team_win', 'first_half_score',
       'second_half_score', 'overtime_score', 'final_score'],
      dtype='object')

## Data ingestion on PostgreSQL DB

In [ ]:
engine = create_engine(
    f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

In [ ]:


primary_keys_map = {
    "dim_championships": ["competition_id"],
    "dim_teams": ["team_id"],
    "dim_players": ["player_id"],
    "fact_matches": ["match_id", "match_round"],
    "fact_teams_stats": ["match_id", "match_round", "team_id"],
    "fact_players_stats": ["match_id", "match_round", "player_id"]
}

def insert_new_records(df, table_name, pk_cols):
    try:
        pk_cols_str = ', '.join(pk_cols)
        query = f"SELECT {pk_cols_str} FROM {table_name}"
        
        existing_pks_df = pd.read_sql(query, engine)

        if len(pk_cols) == 1:
            col = pk_cols[0]
            mask = ~df[col].isin(existing_pks_df[col])
        else:

            df_keys = df[pk_cols].apply(tuple, axis=1)
            existing_keys = existing_pks_df.apply(tuple, axis=1)
            mask = ~df_keys.isin(existing_keys)
        
        new_records_df = df[mask]
        
        if not new_records_df.empty:
            new_records_df.to_sql(
                table_name,
                con=engine,
                if_exists='append',
                index=False,
                method='multi'
            )
            print(f"Inserted {len(new_records_df)} new records into {table_name}")
        else:
            print(f"No new records to insert for {table_name}")
    except SQLAlchemyError as e:
        print(f"Database error while inserting into {table_name}: {e}")
    except Exception as e:
        print(f"Unexpected error while inserting into {table_name}: {e}")

def main():
    try:
        insert_new_records(dim_championships, "dim_championships", primary_keys_map["dim_championships"])
        insert_new_records(dim_teams, "dim_teams", primary_keys_map["dim_teams"])
        insert_new_records(dim_players, "dim_players", primary_keys_map["dim_players"])
        insert_new_records(fact_matches, "fact_matches", primary_keys_map["fact_matches"])
        insert_new_records(fact_teams_stats, "fact_teams_stats", primary_keys_map["fact_teams_stats"])
        insert_new_records(fact_players_stats, "fact_players_stats", primary_keys_map["fact_players_stats"])
    except Exception as e:
        print(f"Error in main loading process: {e}")

if __name__ == "__main__":
    main()


No new records to insert for dim_championships
No new records to insert for dim_teams
Inserted 3 new records into dim_players
Inserted 20 new records into fact_matches
Inserted 5454 new records into fact_teams_stats
Inserted 27257 new records into fact_players_stats
